In [2]:
%load_ext autoreload
%autoreload 2

In [38]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import numpy as np

import util as u

from glob import glob
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_columns', 2000)


In [4]:
#巡回先を作る
#枠とレースIDがあればマージできる？
result_infofile_list = glob("row_html/01/info_html/*") # 後々フォルダーが変わる
main_info_file = [x.replace("\\", "/") for x in result_infofile_list]

result_resultfile_list = glob("test/*/result_html/*") # 後々フォルダーが変わる
main_result_file = [x.replace("\\", "/") for x in result_resultfile_list]

target_info_list = [] #配列を空にする
target_result_list = [] #配列を空にする

match_up_info_list = [] #配列を空にする
match_up_result_list = [] #配列を空にする

#is_file = os.path.isfile("match_up.txt")

#if is_file:
#    with open("match_up.txt", "r") as tf:
#        match_up_list = tf.read().split(',')[:-1]
#    #途中から始めるためのコード書く
#    target_txts_list = set(result_file) - set(match_up_list)
#else:
#    target_txts_list = result_file

target_info_list = main_info_file[0:1]
target_result_list = main_result_file[0:1]

In [5]:
#データフレームを4個作る
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
main_result = pd.DataFrame(index=[])
main_result_dummy = pd.DataFrame(index=[])

#New_col = 'insert_1_1 insert_1_2 insert_2_1 insert_2_2 insert_3_1 insert_3_2 insert_4_1 insert_4_2 insert_5_1 insert_5_2 insert_6_1 insert_6_2 insert_7_1 insert_7_2 \
#            start_timing_1_1 start_timing_1_2 start_timing_2_1 start_timing_2_2 start_timing_3_1 start_timing_3_2 start_timing_4_1 start_timing_4_2 start_timing_5_1 start_timing_5_2 start_timing_6_1 start_timing_6_2 start_timing_7_1 start_timing_7_2 \
#            rank_1_1 rank_1_2 rank_2_1 rank_2_2 rank_3_1 rank_3_2 rank_4_1 rank_4_2 rank_5_1 rank_5_2 rank_6_1 rank_6_2 rank_7_1 rank_7_2'.split()

dfs_all = pd.DataFrame(index=[])
dfs_dummy = pd.DataFrame(index=[])

In [6]:
###後で消す↓###
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
###↑後で消す###

# info_htmlの加工をする
# race_id、枠、選手ID、級別、体重(直前情報から取る)、調整重量、F数、L数、平均ST、モーター2連、モーター3連、初日進入、
# 初日ST、初日成績、～ 、最終日、展示タイム、チルト、プロペラ、部品交換

for target_info in tqdm(target_info_list):
    for i in range(1, 7):
        target_info_day = target_info[-19:-5]
        Pit_No = f"{i:02}"
        
        #レース結果を保存したhtmlファイルとそのDF
        target_info_day = target_info[-19:-5]
        dfs = pd.read_html(target_info, encoding="utf-8")
        race_info1_df = dfs[0]
        
        #dfs[0]の整理
        df_player_info = u.split_inner_data(u.parse(race_info1_df), target_info_day)
        df_player_info = df_player_info.iloc[i-1]

        player_ID = df_player_info["pid"]

        #複勝率、三連率を計算
        player_win_rate = u.player_win_rate_cal(target_info_day, player_ID)
        se_player_win_rate = player_win_rate.iloc[0]

        #開催期間の艇番を表示
        df_bort_color = u.bort_color(target_info).iloc[i-1]
        #開催期間の艇番を表示
        df_open_grades = u.Open_Grades_sort(target_info).iloc[i-1]

        #天気情報を追加
        main_info_dummy = pd.concat([df_player_info, u.weather_condition(target_info)])
        #勝率を追加
        main_info_dummy = pd.concat([main_info_dummy, se_player_win_rate])
        #開催期間の艇番を追加
        main_info_dummy = pd.concat([main_info_dummy, df_bort_color])
        #開催中のレース成績
        main_info_dummy = pd.concat([main_info_dummy, df_open_grades])
        #ダミーの情報をメイン情報に追加
        main_info = main_info.append(main_info_dummy, ignore_index=True)


C:\Users\rstta\AppData\Local\Temp/ipykernel_15732/2234562932.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\rstta\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2947: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [7]:
display(main_info.head(6))

,rid,en,pid,class,branch,weight,F,L,st,mnww,mnwww,idx,temp,whether,win_s,win_a,temp_w,wave_h,q_Total_2,q_Total_3,h_Total_2,h_Total_3,q_Venue_2,q_Venue_3,h_Venue_2,h_Venue_3,bort_color_1_1,bort_color_1_2,bort_color_2_1,bort_color_2_2,bort_color_3_1,bort_color_3_2,bort_color_4_1,bort_color_4_2,bort_color_5_1,bort_color_5_2,bort_color_6_1,bort_color_6_2,bort_color_7_1,bort_color_7_2,insert_1_1,insert_1_2,insert_2_1,insert_2_2,insert_3_1,insert_3_2,insert_4_1,insert_4_2,insert_5_1,insert_5_2,insert_6_1,insert_6_2,insert_7_1,insert_7_2,start_timing_1_1,start_timing_1_2,start_timing_2_1,start_timing_2_2,start_timing_3_1,start_timing_3_2,start_timing_4_1,start_timing_4_2,start_timing_5_1,start_timing_5_2,start_timing_6_1,start_timing_6_2,start_timing_7_1,start_timing_7_2,rank_1_1,rank_1_2,rank_2_1,rank_2_2,rank_3_1,rank_3_2,rank_4_1,rank_4_2,rank_5_1,rank_5_2,rank_6_1,rank_6_2,rank_7_1,rank_7_2
0,01_20140102_01,1.0,3115,B1,群馬,50.0,0,0,0.22,0.00,-,01_20140102_01_01,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01_20140102_01,2.0,3560,B1,群馬,49.0,0,0,0.19,0.00,-,01_20140102_01_02,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01_20140102_01,3.0,3376,B1,群馬,48.5,0,0,0.24,14.20,-,01_20140102_01_03,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01_20140102_01,4.0,3532,A1,群馬,52.8,0,0,0.17,0.00,-,01_20140102_01_04,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01_20140102_01,5.0,2808,B2,群馬,51.4,0,0,0.29,0.00,-,01_20140102_01_05,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01_20140102_01,6.0,4645,B1,群馬,50.4,0,0,0.16,66.60,-,01_20140102_01_06,5.0,晴,8,4,6.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
for main_info in tqdm(main_info_file):

    dfs = pd.read_html(main_info, encoding="utf-8")
    dfs = dfs[1][::4]
    
    output_file = "test.csv"
    dfs.to_csv(os.path.join(output_file), mode="a",header=False, index=False)


C:\Users\rstta\AppData\Local\Temp/ipykernel_15732/684167003.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for main_info in tqdm(main_info_file):


  0%|          | 0/18910 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
for main_info in tqdm(main_info_file):

    dfs = pd.read_html(main_info, encoding="utf-8")
    dfs = dfs[1][::4]
    dfs = dfs.drop("前走成績", axis=1)
    dfs["race_id"] = main_info[22:-5]
    
    output_file = "test.csv"
    dfs.to_csv(os.path.join(output_file), mode="a",header=False, index=False)

C:\Users\rstta\AppData\Local\Temp/ipykernel_15732/630000531.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for main_info in tqdm(main_info_file):


  0%|          | 0/18910 [00:00<?, ?it/s]

C:\Users\rstta\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


KeyboardInterrupt: 

In [10]:
target_info_list = main_info_file

In [11]:
dfs = pd.read_html("test/01/info_html/01_20140103_01.html", encoding="utf-8")[1]
dfs["rid"] = target_info[-19:-5]
dfs = dfs.drop("前走成績", axis=1)
display(dfs.loc[2, ("体重", "調整重量")])
display(dfs[::2][1::2])


'0.0'

,枠,写真,ボートレーサー,体重,展示タイム,チルト,プロペラ,部品交換,rid
,枠,写真,ボートレーサー,調整重量,展示タイム,チルト,プロペラ,部品交換,
2,1,NaN,森下 勇,0.0,6.66,-0.5,ヤ,NaN,01_20140102_01
6,2,NaN,永瀬 巧,0.0,6.69,-0.5,ヤ,NaN,01_20140102_01
10,3,NaN,吉村 茂樹,0.0,6.66,-0.5,ヤ,NaN,01_20140102_01
14,4,NaN,山崎 智也,0.0,6.70,-0.5,ヤ,NaN,01_20140102_01
18,5,NaN,金澤 一洋,0.0,6.68,-0.5,ヤ,ピストン×２ シャフト,01_20140102_01
22,6,NaN,椎名 豊,0.0,6.71,-0.5,ヤ,NaN,01_20140102_01


In [12]:
parts_col = "Adj_weight Open_t tilt pera pis shaft gia cab electro ring silinda calbo".split()
#ギヤ ピストン シャフト キャブ 電気 リング シリンダ キャリアボデー
target_info = "row_html/01/info_html/01_20201218_08.html"
df_parts_info = pd.DataFrame(index=[], columns=parts_col)

dfs = pd.read_html(target_info, encoding="utf-8")[1]
dfs["rid"] = target_info[-19:-5]
dfs = dfs.drop("前走成績", axis=1)

df = dfs[::2][1::2]
df.reset_index(drop=True, inplace=True) 
display(df)
display(df_parts_info)
#display(df.loc[0, "展示タイム"])

C:\Users\rstta\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,枠,写真,ボートレーサー,体重,展示タイム,チルト,プロペラ,部品交換,rid
,枠,写真,ボートレーサー,調整重量,展示タイム,チルト,プロペラ,部品交換,
0,1,NaN,荻野 裕介,0.0,6.80,-0.5,NaN,ピストン×２ リング×４ シリンダ,01_20201218_08
1,2,NaN,杢野 誓良,1.0,6.79,-0.5,NaN,NaN,01_20201218_08
2,3,NaN,谷本 幸司,0.0,6.90,-0.5,NaN,NaN,01_20201218_08
3,4,NaN,原田 智和,0.0,6.83,-0.5,NaN,NaN,01_20201218_08
4,5,NaN,金子 拓矢,0.0,6.87,-0.5,NaN,NaN,01_20201218_08
5,6,NaN,小川 知行,0.0,6.88,-0.5,NaN,NaN,01_20201218_08


,Adj_weight,Open_t,tilt,pera,pis,shaft,gia,cab,electro,ring,silinda,calbo


In [13]:
print(target_info)

row_html/01/info_html/01_20201218_08.html


In [14]:
display(df.loc[0, ("部品交換","部品交換")])
x = df.loc[0,("部品交換", "部品交換")]
x = x.split(" ")
print(x)
display(df.loc[1, ("部品交換","部品交換")])

'ピストン×２ リング×４ シリンダ'

['ピストン×２', 'リング×４', 'シリンダ']


nan

In [83]:
parts_col = "Adj_weight Open_t tilt pera pis shaft gia cab electro ring silinda calbo".split()
#ギヤ ピストン シャフト キャブ 電気 リング シリンダ キャリアボデー
target_info = "row_html/01/info_html/01_20201219_03.html"
df_parts_info = pd.DataFrame(index=[], columns=parts_col)
df_parts_info.loc[0,["pis", "shaft", "gia", "cab", "electro", "ring", "silinda", "calbo"]] = 0

dfs = pd.read_html(target_info, encoding="utf-8")[1]
dfs["rid"] = target_info[-19:-5]
dfs = dfs.drop("前走成績", axis=1)

df = dfs[::2][1::2]
df.reset_index(drop=True, inplace=True) 
display(df)
display(df_parts_info)
#display(df.loc[0, "展示タイム"])

C:\Users\rstta\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,枠,写真,ボートレーサー,体重,展示タイム,チルト,プロペラ,部品交換,rid
,枠,写真,ボートレーサー,調整重量,展示タイム,チルト,プロペラ,部品交換,
0,1,NaN,杢野 誓良,0.5,7.06,-0.5,NaN,NaN,01_20201219_03
1,2,NaN,小川 知行,0.0,7.13,0.0,NaN,NaN,01_20201219_03
2,3,NaN,マイケル田代,0.0,7.14,-0.5,NaN,リング×１,01_20201219_03
3,4,NaN,野添 貴裕,0.0,7.05,-0.5,NaN,NaN,01_20201219_03
4,5,NaN,中村 尊,0.0,6.98,0.0,NaN,NaN,01_20201219_03
5,6,NaN,一瀬 明,0.0,7.07,-0.5,NaN,NaN,01_20201219_03


,Adj_weight,Open_t,tilt,pera,pis,shaft,gia,cab,electro,ring,silinda,calbo
0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [16]:
display(df.loc[2, ("部品交換","部品交換")])
x = df.loc[2,("部品交換", "部品交換")]
x = x.split(" ")
print(x)
display(df.loc[1, ("部品交換","部品交換")])

'リング×１'

['リング×１']


nan

In [69]:
df_parts_info.loc[0, "Adj_weight"] = df.loc[0, ("体重", "調整重量")]
df_parts_info.loc[0, "Open_t"] = df.loc[0, ("展示タイム", "展示タイム")]
df_parts_info.loc[0, "tilt"] = df.loc[0, ("チルト", "チルト")]

#ヤ→ヤ(新)、新を1、その他を0にする
if df.loc[0, ("プロペラ", "プロペラ")] == "ヤ→ヤ(新)" or df.loc[0, ("プロペラ", "プロペラ")] == "新":
    new_propera = df.loc[2,("プロペラ","プロペラ")]
    new_propera = new_propera.replace(f"{new_propera}", "1")
    df_parts_info.loc[0, "pera"] = new_propera
else:
    df_parts_info.loc[0, "pera"] = 0

#部品交換がNaNのとき、交換があるときの処理
if df.loc[0, ("部品交換", "部品交換")] is np.nan:
    for parts_c in range(4,12):
        df_parts_info.iloc[0, parts_c] = 0


#display(df_parts_info)

In [78]:
df.loc[0 ,("部品交換", "部品交換")]


nan

In [81]:
display(df_parts_info.iloc[0,[4,5,6,7,8,9,10,11]])

pis        0
shaft      0
gia        0
cab        0
electro    0
ring       0
silinda    0
calbo      0
Name: 0, dtype: object

In [ ]:
df = pd.read_csv("test.csv", encoding="utf-8", header=None)
df = df[~df[6].isnull()]

pattern_dict = {
    "ヤ":"null",
    "ヤ→ヤ(新)":"null"
}
output = [x for x in df[6] if pattern_dict.get(x)!="null"]
print (output)

['新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新', '新',

### データが取れてないところからデータを取る処理を書く

### 現在開催されているデータを引っ張ってくる

### PyCaretってのを使ってみるよ